# Gravity wave propagation in a stratified fluid

This notebook will demonstrate using Gusto to solve the compressible Euler equations in a 2D (x-z) domain, often called a 'vertical slice'. The variables in this system are the velocity $\textbf{u}$, the dry density $\rho$, and the (virtual dry) potential temperature $\theta$. The fluid is stratified and the background state is in hydrostatic balance. The initial conditions consist of a perturbation to $\theta$ in the centre of the domain. This setup is given in the [1994 paper by Skamarock and Klemp](https://journals.ametsoc.org/view/journals/mwre/122/11/1520-0493_1994_122_2623_eaaotk_2_0_co_2.xml), except for the domain width which has been reduced to a third of the size so that the problem runs fast enough to produce results fairly quickly.

As our standard first step, we begin by importing the required libraries and functions from Firedrake and Gusto:

In [1]:
from gusto import *
from firedrake import (as_vector, SpatialCoordinate, PeriodicIntervalMesh,
                       ExtrudedMesh, exp, sin, pi, Function)

INFO     Running /Users/JS1075/firedrake_dec24/lib/python3.12/site-packages/ipykernel_launcher.py -f /private/var/folders/f0/llvlmlb50qg6mmlxs8m6d6d00000gp/T/tmp84ie5eqf.json --HistoryManager.hist_file=:memory:


For a vertical slice domain, we first define a periodic interval mesh  with width equal to that of the required domain and the number of cells equal to the number of columns we require in our final mesh. The periodic mesh is then extruded in the vertical direction by specifying the number of layers and the layer height. We now have a two dimensional mesh with vertically aligned columns.

In [2]:
ncolumns = 50  # number of columns
domain_width = 1.0e5
m = PeriodicIntervalMesh(ncolumns, domain_width)

nlayers = 10  # number of horizontal layers
domain_height = 1.0e4  # Height position of the model top
mesh = ExtrudedMesh(m, layers=nlayers, layer_height=domain_height/nlayers)

We can now set up the domain, physical parameters and equations. We will use the default values for any physical parameters (which can be found [here](https://www.firedrakeproject.org/gusto-docs/gusto.core.html#gusto.core.configuration.CompressibleParameters)). The compatible finite element function spaces are specified by the `family` and `degree` parameters passed to `Domain`. We also specify the timestep, `dt` and pass that to `Domain`.

In [3]:
dt = 6
domain = Domain(mesh, dt=dt, family="CG", degree=1)
parameters = CompressibleParameters(mesh)
eqns = CompressibleEulerEquations(domain, parameters)

We now set up the `OutputParameters` class, specifying the path to the output directory `dirname` and the output frequency `dumpfreq`, set to every 10 timesteps. We would like to visualise the evolution of the potential temperature perturbation, so we specify that as a diagnostic field.

In [4]:
dirname = 'gravity_wave'
output = OutputParameters(dirname=dirname, dumpfreq=10)
diagnostic_fields = [Perturbation('theta')]
io = IO(domain, output, diagnostic_fields=diagnostic_fields)

We will set up a `SemiImplicitQuasiNewton` timestepper. This method splits the equation into terms that are treated explicitly and those that are treated semi-implicitly. Transport and forcing terms are treated separately. We have to specify the transport scheme for each field. Below we use an explicit strong stability preserving RK3 (`SSPRK3`) method for all fields, which we subcycle to keep the Courant number below 0.25 for stability. We apply the `DGUpwind` method to the transport terms for all fields, and additionally use streamline upwind Petrov-Galerkin (SUPG) for the potential temperature.

In [5]:
theta_opts = SUPGOptions()
subcycling_options = SubcyclingOptions(subcycle_by_courant=0.25)

transported_fields = [
    SSPRK3(domain, "u", subcycling_options=subcycling_options),
    SSPRK3(
        domain, "rho", subcycling_options=subcycling_options,
        rk_formulation=RungeKuttaFormulation.linear
    ),
    SSPRK3(
        domain, "theta", subcycling_options=subcycling_options,
        options=theta_opts
    )
]
transport_methods = [
    DGUpwind(eqns, "u"),
    DGUpwind(eqns, "rho", advective_then_flux=True),
    DGUpwind(eqns, "theta", ibp=theta_opts.ibp)
]

linear_solver = CompressibleSolver(eqns)

stepper = SemiImplicitQuasiNewton(
    eqns, io, transported_fields, transport_methods,
    linear_solver=linear_solver
)

INFO     Physical parameters that take non-default values:


INFO     L_v0: 2500000.0, N: 0.01, R_d: 287.0, R_v: 461.0, T_0: 273.15, c_pl: 4186.0, c_pv: 1885.0, c_vv: 1424.0, cp: 1004.5, cv: 717.5, g: 9.810616, kappa: 0.2857142857142857, p_0: 100000.0, w_sat1: 380.3, w_sat2: -17.27, w_sat3: 35.86, w_sat4: 610.9


Now we can initialise the model and set up the background profiles in hydrostatic balance.

In [6]:
# Initial conditions
u0 = stepper.fields("u")
rho0 = stepper.fields("rho")
theta0 = stepper.fields("theta")

# spaces
Vt = domain.spaces("theta")
Vr = domain.spaces("DG")

# Thermodynamic constants required for setting initial conditions
# and reference profiles
g = parameters.g
N = parameters.N
Tsurf = 300.

xz = SpatialCoordinate(mesh)

# N^2 = (g/theta)dtheta/dz => dtheta/dz = theta N^2g => theta=theta_0exp(N^2gz)
thetab = Tsurf*exp(N**2*xz[1]/g)

theta_b = Function(Vt).interpolate(thetab)
rho_b = Function(Vr)

This next step takes the temperature profile and computes the corresponding density that will enforce a hydrostatic balance.

In [7]:
compressible_hydrostatic_balance(eqns, theta_b, rho_b)

We now set up the initial potential temperature profile. We apply a perturbation $\theta'$ to the background balanced $\theta_b$ defined above, where $\theta'$ is given by:
$$\theta' = \Delta \theta \frac{\sin\big(\frac{\pi z}{H}\big)}{1 + \frac{1}{a^2}(x-x_c)^2}$$

In [8]:
a = 5.0e3
deltaTheta = 1.0e-2
theta_pert = deltaTheta*sin(pi*xz[1]/domain_height)/(1 + (xz[0] - domain_width/2)**2/a**2)
theta0.interpolate(theta_b + theta_pert)
rho0.assign(rho_b)
u0.project(as_vector([20.0, 0.0]))

stepper.set_reference_profiles([('rho', rho_b), ('theta', theta_b)])

Running the full simulation will take a while, but you can replace `tmax=2*dt` in the cell below with `tmax=tmax` if you would like to do so.

In [9]:
stepper.run(t=0, tmax=150*dt)

INFO     


INFO     ========================================


INFO     at start of timestep 1, t=0.0, dt=6.0


INFO     Max Courant: 6.00e-02


INFO     Max Courant horizontal: 6.00e-02


...

INFO     Compressible linear solver: hybridized solve


INFO     Compressible linear solver: restore continuity


INFO     Compressible linear solver: theta solve


INFO     TIMELOOP complete. t=900.00000, tmax=900.00000


    Residual norms for ImplicitSolver_ solve.
    0 KSP none resid norm 1.538402593527e+02 true resid norm 1.523301778768e+02 ||r(i)||/||b|| 9.901840943174e-01
    1 KSP none resid norm 1.916111656391e-06 true resid norm 1.916111656391e-06 ||r(i)||/||b|| 1.245520297777e-08
    Residual norms for ImplicitSolver_ solve.
    0 KSP none resid norm 1.551945108770e+02 true resid norm 3.090212892296e+02 ||r(i)||/||b|| 1.991186978737e+00
    1 KSP none resid norm 1.921947184748e-06 true resid norm 1.921947184748e-06 ||r(i)||/||b|| 1.238411831635e-08
    Residual norms for ImplicitSolver_ solve.
    0 KSP none resid norm 2.371769600390e+02 true resid norm 3.668218890188e+02 ||r(i)||/||b|| 1.546616876102e+00
    1 KSP none resid norm 8.822714911214e-14 true resid norm 8.822714911214e-14 ||r(i)||/||b|| 3.719887003259e-16
    Residual norms for ImplicitSolver_ solve.
    0 KSP none resid norm 3.130190751016e+00 true resid norm 2.393715386165e+02 ||r(i)||/||b|| 7.647186949832e+01
    1 KSP none res

Running the reduced domain setup in this notebook will produce vtu files and plotting the final theta perturbation using Paraview should reproduce the figure below.
![gw](figures/gw_final.png)

The figures below are from a simulation using the full domain and at higher resolution. First we show the initial theta perturbation and theta field, followed by the final theta perturbation plotted in the vertical slice and as a cross section through the domain at z=domain_height/2.

![gw_init](../figures/compressible_euler/skamarock_klemp_nonhydrostatic_initial.png)

![gw_init](../figures/compressible_euler/skamarock_klemp_nonhydrostatic_final.png)